Source: https://www.youtube.com/watch?v=SQ1iIKs190Q&list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&index=9

In [ ]:
from kapre.composed import get_melspectrogram_layer
import pickle
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.activations import relu, softmax
from typing import Tuple
import utils.constants as consts

#### Load data from previous notebook

In [ ]:
def build_vgg_model() -> Sequential:
    """
    :return:
    """

    input_shape: Tuple[int, int] = (consts.SAMPLE_RATE, 1.0,)  # Mono audio so 1 channel.

    mel_layer: Sequential = get_melspectrogram_layer(
        input_shape=input_shape,
        n_fft=consts.NUM_FFT,
        hop_length=consts.MEL_HOP_LEN,
        sample_rate=consts.SAMPLE_RATE,
        return_decibel=True,
        input_data_format='channels_last',
        output_data_format='channels_last'
    )

    model = Sequential()

    model.add(mel_layer)
    model.add(Conv2D(
        16,
        (3, 3),
        strides=(1, 1),
        padding=2,
        kernel_size=3
    ))

    model.add(Conv2D)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Softmax
from kapre import STFT, Magnitude, MagnitudeToDecibel
from kapre.composed import get_melspectrogram_layer, get_log_frequency_spectrogram_layer

# 6 channels (!), maybe 1-sec audio signal, for an example.
input_shape = (44100, 6)
sr = 44100

model = Sequential()

# A STFT layer
model.add(STFT(n_fft=2048, win_length=2018, hop_length=1024,
               window_name=None, pad_end=False,
               input_data_format='channels_last', output_data_format='channels_last',
               input_shape=input_shape))
model.add(Magnitude())
model.add(MagnitudeToDecibel())  # these three layers can be replaced with get_stft_magnitude_layer()

# Alternatively, you may want to use a melspectrogram layer
# melgram_layer = get_melspectrogram_layer()
# or log-frequency layer
# log_stft_layer = get_log_frequency_spectrogram_layer()

# add more layers as you want
model.add(Conv2D(32, (3, 3), strides=(2, 2)))
model.add(BatchNormalization())
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(10))
model.add(Softmax())

# Compile the model
model.compile('adam', 'categorical_crossentropy')  # if single-label classification

# train it with raw audio sample inputs
# for example, you may have functions that load your data as below.
x = load_x()  # e.g., x.shape = (10000, 6, 44100)
y = load_y()  # e.g., y.shape = (10000, 10) if it's 10-class classification
# then..
model.fit(x, y)
# Done!